In [51]:
import requests
import pandas as pd
from pandas import json_normalize

# Set your API key
api_key = '47a596f3e23938512be3339f15c55adf'

# API endpoint for detailed property information
url_detail = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/property/detail"
# API endpoint for searching properties within a ZIP code
url_snapshot = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/property/snapshot"

# Prompt the user for address details
address1 = input("Please enter the street address (e.g., 4529 Winona Court): ")
address2 = input("Please enter the city and state (e.g., Denver, CO): ")

# Parameters for detail request
params_detail = {
    'address1': address1,
    'address2': address2
}

# Headers for the API requests
headers = {
    'accept': "application/json",
    'apikey': api_key,
}

# Make the API request for detailed property information
response_detail = requests.get(url_detail, headers=headers, params=params_detail)

# Process the detailed property response
if response_detail.status_code == 200:
    data = response_detail.json()
    properties = json_normalize(data['property'], errors='ignore')
    if not properties.empty:
        # Select available columns dynamically
        available_columns = properties.columns
        selected_columns = [col for col in ['address.oneLine', 'building.rooms.bathstotal', 'building.size.livingsize', 'summary.yearbuilt'] if col in available_columns]
        properties_display = properties[selected_columns]
        column_names = {
            'address.oneLine': 'Address',
            'building.rooms.bathstotal': 'Total Bathrooms',
            'building.size.livingsize': 'Living Area (sq ft)',
            'summary.yearbuilt': 'Year Built'
        }
        properties_display.rename(columns=column_names, inplace=True)
        print("Detailed Property Information:")
        print(properties_display)
    else:
        print("No property details found for the given address.")
else:
    print(f"Failed to fetch detailed property info: {response_detail.status_code}")

# Prompt for ZIP code input or use from detailed response if available
zip_code = properties['address.postal1'].iloc[0] if not properties.empty else input("Please enter the ZIP code for property search: ")

# Parameters for snapshot request using user-provided ZIP code
params_snapshot = {
    'postalcode': zip_code,
    'pagesize': 10,  # Adjust as needed
    'page': 1
}

# Make the API request to find properties in the same ZIP code
response_snapshot = requests.get(url_snapshot, headers=headers, params=params_snapshot)

if response_snapshot.status_code == 200:
    snapshot_data = response_snapshot.json()
    snapshot_properties = json_normalize(snapshot_data['property'], errors='ignore')
    if not snapshot_properties.empty:
        # Select available columns dynamically
        snapshot_columns = snapshot_properties.columns
        snapshot_selected_columns = [col for col in ['address.oneLine', 'building.size.livingsize', 'summary.yearbuilt'] if col in snapshot_columns]
        snapshot_display = snapshot_properties[snapshot_selected_columns]
        snapshot_column_names = {
            'address.oneLine': 'Address',
            'building.size.livingsize': 'Living Area (sq ft)',
            'summary.yearbuilt': 'Year Built'
        }
        snapshot_display.rename(columns=snapshot_column_names, inplace=True)
        print(f"Properties in Area {zip_code}:")
        print(snapshot_display)
    else:
        print("No additional properties found in the given ZIP code.")
else:
    print(f"Failed to fetch properties in ZIP code: {response_snapshot.status_code}")


Please enter the street address (e.g., 4529 Winona Court): 4529 Winona Court
Please enter the city and state (e.g., Denver, CO): Denver, CO


<ipython-input-51-db128b67abab>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_display.rename(columns=column_names, inplace=True)


Detailed Property Information:
                            Address  Total Bathrooms  Living Area (sq ft)  \
0  4529 WINONA CT, DENVER, CO 80212              1.0                  934   

   Year Built  
0        1900  
Properties in Area 80212:
                                 Address  Year Built
0       5316 STUART ST, DENVER, CO 80212        2002
1        4466 PERRY ST, DENVER, CO 80212        1906
2   3822 SHERIDAN BLVD, DENVER, CO 80212        1922
3        4160 YATES ST, DENVER, CO 80212        1922
4  5525 CLEAR CREEK DR, DENVER, CO 80212        1960
5      4860 QUITMAN ST, DENVER, CO 80212        1921
6        2836 VRAIN ST, DENVER, CO 80212        1932
7      4091 W 53RD AVE, DENVER, CO 80212        1942
8        4242 PERRY ST, DENVER, CO 80212        1953
9        3635 WOLFF ST, DENVER, CO 80212        1960


<ipython-input-51-db128b67abab>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snapshot_display.rename(columns=snapshot_column_names, inplace=True)
